# Advent of Code 2022

## Day 12: Hill Climbing Algorithm

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

I implmented [Dijkstra's shortest path algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm). Kinda, I didn't bother with an efficient implementation of a heap and just used a heap based on a sorted list. The graph is small enough that this doesn't matter, but an obvious implement is replacing this with a [binary heap](https://en.wikipedia.org/wiki/Binary_heap) so don't use this code for anything that needs ot be efficient. Also I don't need [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) again because it's small enough, also A* wouldn't have worked for part 2, only part 1.

This wasn't difficult to think of since I'm very familiar with using graph search in my day job.

In part 2 we don't have a target node but rather a target value, so I added an option to pass either `target`: or `target_height` to the search.

Once we get the path, the solution is just the path length - 1.

### Imports

In [ ]:
import string
import numpy as np
from typing import Optional

### Decoding the File

In [ ]:
SYMBOLS = 'S' + string.ascii_lowercase + 'E'

def decode_map(filename):
    rv = []
    with open(filename) as file:
        for line in file:
            current = []
            line = line.strip()
            for char in line:
                current.append(SYMBOLS.index(char))
            rv.append(current)
    return np.array(rv)

def locate(arr, value):
    height, width = arr.shape
    for y in range(height):
        for x in range(width):
            if arr[y,x] == value:
                return y, x
    raise ValueError(f'{value} not found in arr')

In [ ]:
INPUT_FILE = 'data/input12.txt'

### Finding the Neighbours of a Node

In [ ]:
def in_bound(arr, node):
    height, width = arr.shape
    y, x = node
    return 0 <= y < height and 0 <= x < width

def unconditional_neighbours(node):
    y, x = node
    yield y+1, x
    yield y-1, x
    yield y, x+1
    yield y, x-1

def neighbours(arr, node):
    v = arr[node]
    rv = []
    for y, x in unconditional_neighbours(node):
        if in_bound(arr, (y, x)) and arr[y, x] <= v+1:
            rv.append((y, x))
    return rv

### Different Neighbour Function for Part 2

Just applies the neighbour rule backwards.

In [ ]:
def reverse_neighbours(arr, node):
    v = arr[node]
    rv = []
    for y, x in unconditional_neighbours(node):
        if in_bound(arr, (y, x)):
            to_height = v
            from_height = arr[y, x]
            if to_height <= from_height + 1:
               rv.append((y, x))
    return rv

### Heap

Don't use this implementation of a heap in a real application, it's not efficient. Good enough for the puzzle.

In [ ]:
class Heap:

    __slots__ = ['data']

    def __init__(self):
        self.data = []

    def add(self, element: tuple[int, int], key: int):
        for k, e in self.data:
            if e == element:
                raise ValueError(f'duplicate element: {element}')
        self.data.append((key, element))
        self.data.sort()

    def decrease_key(self, element: tuple[int, int], key: int):
        index = None
        for i, e in enumerate(self.data):
            if e[1] == element:
                index = i
        if index is None:
            raise ValueError(f'no such element: {element}')
        old_key, _ = self.data[index]
        if old_key < key:
            raise ValueError(f'cannot increase key for {element} from {old_key} to {key}')
        del self.data[index]
        self.add(element, key)

    def delete_minimum(self):
        if not self:
            raise ValueError(f'no such element')
        _, rv = self.data[0]
        del self.data[0]
        return rv

    def __bool__(self):
        return bool(self.data)

    def __str__(self):
        return '\n'.join([f'{key}: {element}' for key, element in self.data])

    def __repr__(self):
        return str(self)

### Dijkstra's Shortest Path Algorithm



In [ ]:
def contruct_path(prev, source, target):
    rv = [target]
    while rv[0] != source:
        p = prev[rv[0]]
        rv = [p] + rv
    return rv

def search(arr: np.ndarray,
           source: tuple[int, int],
           target: Optional[tuple[int, int]],
           target_height: Optional[int]):

    previous_vertices = {} # to reconstruct the path

    frontier = Heap()
    frontier.add(source, 0)
    distances = {source: 0}

    while frontier:

        current = frontier.delete_minimum()
        old_total_distance = distances[current]

        # part 1
        if target is not None:

            # if done, path from source to target
            if current == target:
                return contruct_path(previous_vertices, source, target)

            # standard neighbour function
            neighbour_function = neighbours

        # part 2
        else:

            # if done, path from source to current
            if arr[current] == target_height:
                return contruct_path(previous_vertices, source, current)

            # reverse neighbour function
            neighbour_function = reverse_neighbours

        for neighbour in neighbour_function(arr, current):

            new_total_distance = old_total_distance + 1
            known_distance = None
            if neighbour in distances:
                known_distance = distances[neighbour]

            # newly discovered node
            if not known_distance:
                distances[neighbour] = new_total_distance
                frontier.add(neighbour, new_total_distance)
                previous_vertices[neighbour] = current

            # previously discovered node
            elif new_total_distance < known_distance:
                distances[neighbour] = new_total_distance
                frontier.decrease_key(neighbour, new_total_distance)
                previous_vertices[neighbour] = current

    raise ValueError(f'No path found!!!')

### Part 1

In [ ]:
def main():

    map = decode_map(INPUT_FILE)
    start = locate(map, SYMBOLS.index('S'))
    end = locate(map, SYMBOLS.index('E'))

    path = search(map, start, end, None)
    num_vertices = len(path)
    num_edges = num_vertices - 1

    print(f'The number of steps taken from start to end is {num_edges}.')

In [ ]:
if __name__ == '__main__':
    main()

part 2

In [ ]:
def main():

    map = decode_map(INPUT_FILE)
    end = locate(map, SYMBOLS.index('E'))

    path = search(map, end, target=None, target_height=1)
    num_vertices = len(path)
    num_edges = num_vertices - 1

    print(f'The number of steps taken from any height 1 to end is {num_edges}.')

In [ ]:
if __name__ == '__main__':
    main()